In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()
messages=[
    { 'role': 'user', 'content': "Сгенерируй код сортировки пузырьком на Python"}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


```python
def bubble_sort(numbers):
    for i in range(len(numbers)):
        for j in range(len(numbers) - 1):
            if numbers[j] > numbers[j + 1]:
                numbers[j], numbers[j + 1] = numbers[j + 1], numbers[j]
    return numbers

numbers = [64, 34, 25, 12, 22, 11, 90]
print(bubble_sort(numbers))
```



In [5]:
model 

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )

In [6]:
import numpy as np 
print(f'num params before SVD compression = {sum( [ np.prod(item.size())  for item in model.parameters() ])}')

num params before SVD compression = 6740512768


In [7]:
!nvidia-smi

Tue Aug  6 11:13:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:83:00.0 Off |                    0 |
| N/A   31C    P0             69W /  400W |   25064MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
import math
from typing import Optional, Tuple

import torch
import torch.utils.checkpoint
from torch import nn

from transformers.activations import ACT2FN
from transformers.utils import logging
from transformers import LlamaConfig

logger = logging.get_logger(__name__)

_CONFIG_FOR_DOC = "LlamaConfig"

class LlamaRMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        LlamaRMSNorm is equivalent to T5LayerNorm
        """
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        variance = hidden_states.to(torch.float32).pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)

        # convert into half-precision if necessary
        if self.weight.dtype in [torch.float16, torch.bfloat16]:
            hidden_states = hidden_states.to(self.weight.dtype)

        return self.weight * hidden_states


class LlamaRotaryEmbedding(torch.nn.Module):
    def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None):
        super().__init__()
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float().to(device) / dim))
        self.register_buffer("inv_freq", inv_freq)

        # Build here to make `torch.jit.trace` work.
        self.max_seq_len_cached = max_position_embeddings
        t = torch.arange(self.max_seq_len_cached, device=self.inv_freq.device, dtype=self.inv_freq.dtype)
        freqs = torch.einsum("i,j->ij", t, self.inv_freq)
        # Different from paper, but it uses a different permutation in order to obtain the same calculation
        emb = torch.cat((freqs, freqs), dim=-1)
        self.register_buffer("cos_cached", emb.cos()[None, None, :, :], persistent=False)
        self.register_buffer("sin_cached", emb.sin()[None, None, :, :], persistent=False)

    def forward(self, x, seq_len=None):
        # x: [bs, num_attention_heads, seq_len, head_size]
        # This `if` block is unlikely to be run after we build sin/cos in `__init__`. Keep the logic here just in case.
        if seq_len > self.max_seq_len_cached:
            self.max_seq_len_cached = seq_len
            t = torch.arange(self.max_seq_len_cached, device=x.device, dtype=self.inv_freq.dtype)
            freqs = torch.einsum("i,j->ij", t, self.inv_freq)
            # Different from paper, but it uses a different permutation in order to obtain the same calculation
            emb = torch.cat((freqs, freqs), dim=-1).to(x.device)
            self.register_buffer("cos_cached", emb.cos()[None, None, :, :], persistent=False)
            self.register_buffer("sin_cached", emb.sin()[None, None, :, :], persistent=False)
        return (
            self.cos_cached[:, :, :seq_len, ...].to(dtype=x.dtype),
            self.sin_cached[:, :, :seq_len, ...].to(dtype=x.dtype),
        )


def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, position_ids):
    gather_indices = position_ids[:, None, :, None]  # [bs, 1, seq_len, 1]
    gather_indices = gather_indices.repeat(1, cos.shape[1], 1, cos.shape[3])
    cos = torch.gather(cos.repeat(gather_indices.shape[0], 1, 1, 1), 2, gather_indices)
    sin = torch.gather(sin.repeat(gather_indices.shape[0], 1, 1, 1), 2, gather_indices)
    
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


class SVD_LlamaMLP(nn.Module):
    def __init__(
        self,
        hidden_size: int,
        intermediate_size: int,
        hidden_act: str,
        ratio=1
    ):
        super().__init__()
        self.ratio = ratio
        low_rank = int(intermediate_size * hidden_size * self.ratio / (intermediate_size + hidden_size))
        self.gate_u_proj = nn.Linear(low_rank, intermediate_size, bias=False)
        self.gate_v_proj = nn.Linear(hidden_size, low_rank, bias=False)
        
        self.down_u_proj = nn.Linear(low_rank, hidden_size, bias=False)
        self.down_v_proj = nn.Linear(intermediate_size, low_rank, bias=False)
        
        self.up_u_proj = nn.Linear(low_rank, intermediate_size, bias=False)
        self.up_v_proj = nn.Linear(hidden_size, low_rank, bias=False)
        self.act_fn = ACT2FN[hidden_act]

    def forward(self, x):
        up = self.up_u_proj(self.up_v_proj(x))
        gate = self.gate_u_proj(self.gate_v_proj(x))
        return self.down_u_proj(self.down_v_proj(self.act_fn(gate) * up))


class SVD_LlamaAttention(nn.Module):
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config: LlamaConfig, ratio=1):
        super().__init__()
        
        self.config = config
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.hidden_size // self.num_heads
        self.max_position_embeddings = config.max_position_embeddings
        self.ratio = ratio # 1 means no truncate, just keep normal attn

        if (self.head_dim * self.num_heads) != self.hidden_size:
            raise ValueError(
                f"hidden_size must be divisible by num_heads (got `hidden_size`: {self.hidden_size}"
                f" and `num_heads`: {self.num_heads})."
            )
        low_rank = int(self.hidden_size * self.ratio/2)
        self.q_u_proj = nn.Linear(low_rank, self.num_heads * self.head_dim, bias=False)
        self.q_v_proj = nn.Linear(self.hidden_size, low_rank, bias=False)

        self.k_u_proj = nn.Linear(low_rank, self.num_heads * self.head_dim, bias=False)
        self.k_v_proj = nn.Linear(self.hidden_size, low_rank, bias=False)

        self.v_u_proj = nn.Linear(low_rank, self.num_heads * self.head_dim, bias=False)
        self.v_v_proj = nn.Linear(self.hidden_size, low_rank, bias=False)

        self.o_u_proj = nn.Linear(low_rank, self.hidden_size, bias=False)
        self.o_v_proj = nn.Linear(self.num_heads * self.head_dim, low_rank, bias=False)

        self.rotary_emb = LlamaRotaryEmbedding(self.head_dim, max_position_embeddings=self.max_position_embeddings)

    def _shape(self, tensor: torch.Tensor, seq_len: int, bsz: int):
        return tensor.view(bsz, seq_len, self.num_heads, self.head_dim).transpose(1, 2).contiguous()

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        output_attentions: bool = False,
        use_cache: bool = False,
        cache_position: torch.Tensor = None
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        bsz, q_len, _ = hidden_states.size()
    
        query_states = self.q_u_proj(self.q_v_proj(hidden_states)).view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)

        key_states = self.k_u_proj(self.k_v_proj(hidden_states)).view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)

        value_states = self.v_u_proj(self.v_v_proj(hidden_states)).view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)

        kv_seq_len = key_states.shape[-2]
        if past_key_value is not None:
            kv_seq_len += past_key_value[0].shape[-2]
        cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)
 
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)
        # [bsz, nh, t, hd]

        if past_key_value is not None:
            # reuse k, v, self_attention
            key_states = torch.cat([past_key_value[0], key_states], dim=2)
            value_states = torch.cat([past_key_value[1], value_states], dim=2)

        past_key_value = (key_states, value_states) if use_cache else None

        attn_weights = torch.matmul(query_states, key_states.transpose(2, 3)) / math.sqrt(self.head_dim)

        if attn_weights.size() != (bsz, self.num_heads, q_len, kv_seq_len):
            raise ValueError(
                f"Attention weights should be of size {(bsz * self.num_heads, q_len, kv_seq_len)}, but is"
                f" {attn_weights.size()}"
            )

        if attention_mask is not None:
            if attention_mask.size() != (bsz, 1, q_len, kv_seq_len):
                raise ValueError(
                    f"Attention mask should be of size {(bsz, 1, q_len, kv_seq_len)}, but is {attention_mask.size()}"
                )
            attn_weights = attn_weights + attention_mask
            attn_weights = torch.max(attn_weights, torch.tensor(torch.finfo(attn_weights.dtype).min, device=attn_weights.device))

        # upcast attention to fp32
        attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)
        attn_output = torch.matmul(attn_weights, value_states)

        if attn_output.size() != (bsz, self.num_heads, q_len, self.head_dim):
            raise ValueError(
                f"`attn_output` should be of size {(bsz, self.num_heads, q_len, self.head_dim)}, but is"
                f" {attn_output.size()}"
            )

        attn_output = attn_output.transpose(1, 2)
        attn_output = attn_output.reshape(bsz, q_len, -1)

        attn_output = self.o_u_proj(self.o_v_proj(attn_output))

        if not output_attentions:
            attn_weights = None

        return attn_output, attn_weights, past_key_value

In [9]:
def find_layers(module, layers=[nn.Conv2d, nn.Linear], name=''):
    if type(module) in layers:
        return {name: module}
    res = {}
    for name1, child in module.named_children():
        res.update(find_layers(
            child, layers=layers, name=name + '.' + name1 if name != '' else name1
        ))
    return res

In [10]:
from tqdm import tqdm

In [11]:
ratio = 0.8

In [8]:
def hook(module, input, output):
    inp = input[0].detach().float()
    if inp.dim() == 2:   # for opt
        inp = inp.unsqueeze(0)
    adds = torch.matmul(inp.transpose(1,2), inp)
    adds_sum = torch.sum(adds, dim=0)
    module.raw_scaling_diag_matrix += adds_sum
    del inp, adds, adds_sum
    torch.cuda.empty_cache()

In [19]:
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        print(f'{name} weight shape = {module.weight.data.shape}')
        print(f'{name} rank = {torch.linalg.matrix_rank( module.weight.data.float()) }')
        print()
        #module.raw_scaling_diag_matrix = 0
        #module.register_forward_hook(hook)

model.layers.0.self_attn.q_proj weight shape = torch.Size([4096, 4096])
model.layers.0.self_attn.q_proj rank = 3118

model.layers.0.self_attn.k_proj weight shape = torch.Size([4096, 4096])
model.layers.0.self_attn.k_proj rank = 3260

model.layers.0.self_attn.v_proj weight shape = torch.Size([4096, 4096])
model.layers.0.self_attn.v_proj rank = 4072

model.layers.0.self_attn.o_proj weight shape = torch.Size([4096, 4096])
model.layers.0.self_attn.o_proj rank = 4079

model.layers.0.mlp.gate_proj weight shape = torch.Size([11008, 4096])
model.layers.0.mlp.gate_proj rank = 4096

model.layers.0.mlp.up_proj weight shape = torch.Size([11008, 4096])
model.layers.0.mlp.up_proj rank = 4096

model.layers.0.mlp.down_proj weight shape = torch.Size([4096, 11008])
model.layers.0.mlp.down_proj rank = 4096

model.layers.1.self_attn.q_proj weight shape = torch.Size([4096, 4096])
model.layers.1.self_attn.q_proj rank = 3313

model.layers.1.self_attn.k_proj weight shape = torch.Size([4096, 4096])
model.layer

In [10]:
from datasets import load_dataset
from tqdm import trange
def get_wikitext2(nsamples, seed, seqlen, tokenizer, dataset_cache_dir=None):
    traindata = load_dataset('nickrosh/Evol-Instruct-Code-80k-v1', split='train', cache_dir=dataset_cache_dir)
    trainloader = [tokenizer(item, return_tensors='pt') for item in traindata['instruction'][:256]]
    return trainloader

In [11]:
calib_loader = get_wikitext2(nsamples = 256, seed = 42, seqlen = 2024, tokenizer = tokenizer, dataset_cache_dir=None)

In [12]:
len(calib_loader)

256

In [13]:
for batch in tqdm(calib_loader):
    print(f'batch len = {len(batch)}')
    #batch = {k: v for k, v in batch.items()}
    with torch.inference_mode(): 
        model(**batch.to(model.device))

  0%|          | 0/256 [00:00<?, ?it/s]

batch len = 2


  0%|          | 1/256 [00:00<03:45,  1.13it/s]

batch len = 2


  1%|          | 2/256 [00:02<05:06,  1.21s/it]

...

 99%|█████████▉| 254/256 [06:18<00:03,  1.50s/it]

batch len = 2


100%|█████████▉| 255/256 [06:19<00:01,  1.50s/it]

batch len = 2


100%|██████████| 256/256 [06:21<00:00,  1.49s/it]


In [14]:
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        module._forward_hooks.clear()

for i in trange(len(model.model.layers)):
    subset = find_layers(model.model.layers[i])
    for name in subset:
        subset[name].raw_scaling_diag_matrix = subset[name].raw_scaling_diag_matrix.cpu()

100%|██████████| 32/32 [00:03<00:00,  8.23it/s]


In [19]:
print(1) 

1


In [ ]:
dev = 'cuda'

In [ ]:
profiling_mat = {}
print("Start Cholesky Decomposition...")
for i in tqdm(range(len(model.model.layers))):
    layer_profile = {}
    subset = find_layers(model.model.layers[i])
    for name in subset:
        raw_scaling_diag_matrix = subset[name].raw_scaling_diag_matrix.double().to(dev)
        
        try:
            scaling_diag_matrix = torch.linalg.cholesky(raw_scaling_diag_matrix)
        except Exception as e:
            print("Warning: eigen scaling_diag_matrix is not positive!")
            eigenvalues = torch.linalg.eigvalsh(raw_scaling_diag_matrix)
            raw_scaling_diag_matrix += (- eigenvalues[0] + 1e-6) * torch.eye(raw_scaling_diag_matrix.shape[0]).to(dev)
            scaling_diag_matrix = torch.linalg.cholesky(raw_scaling_diag_matrix)
            eigenvalues = None
            del eigenvalues
            
        layer_profile[name] = scaling_diag_matrix.cpu()
        scaling_diag_matrix = raw_scaling_diag_matrix = subset[name].raw_scaling_diag_matrix = None
        del scaling_diag_matrix, raw_scaling_diag_matrix, subset[name].raw_scaling_diag_matrix
        torch.cuda.empty_cache()
    profiling_mat[i] = layer_profile

In [21]:
#import pickle 

#with open('profiling_mat_deepseek_1.3b.pickle', 'wb') as f: 
 #   pickle.dump(profiling_mat, f)

In [33]:
#profiling_mat

{0: {'self_attn.q_proj': tensor([[16.2740,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.2524,  7.8340,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.7396,  1.6774, 29.5155,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 2.0125, -2.1906,  1.7948,  ...,  7.5375,  0.0000,  0.0000],
          [-2.1371, -1.8694, -0.2002,  ..., -0.1870,  6.2662,  0.0000],
          [-0.4757, -1.7651, -2.2655,  ..., -0.0669,  0.2023,  3.0352]],
         dtype=torch.float64),
  'self_attn.k_proj': tensor([[16.2740,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.2524,  7.8340,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.7396,  1.6774, 29.5155,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 2.0125, -2.1906,  1.7948,  ...,  7.5375,  0.0000,  0.0000],
          [-2.1371, -1.8694, -0.2002,  ..., -0.1870,  6.2662,  0.0000],
          [-0.4757, -1.7651, -2.2655,  ..., -0.0669,  0.2023,  3.0352]],
         dtype=torch.float64),

In [ ]:
#import pickle 

#with open('profiling_mat_deepseek_1.3b.pickle', 'rb') as f: 
#    profiling_mat = pickle.load(f)

In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
!nvidia-smi

Mon Aug  5 08:27:49 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:83:00.0 Off |                    0 |
| N/A   34C    P0             86W /  400W |   27972MiB /  81920MiB |     48%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from tqdm import trange
for i in trange(len(model.model.layers)):
    layer = model.model.layers[i]
    subset = find_layers(layer)

    svd_attn = SVD_LlamaAttention(config=model.config, ratio=ratio)
    svd_mlp = SVD_LlamaMLP(hidden_size=layer.hidden_size, intermediate_size=model.config.intermediate_size, hidden_act=model.config.hidden_act, ratio=ratio)

    for name in subset:
        W = subset[name].weight.data.float()
        dtype = W.dtype
        scaling_diag_matrix = profiling_mat[i][name].cuda()
        try:
            scaling_matrix_inv = torch.linalg.inv(scaling_diag_matrix)
        except Exception as e:
            print("Warning: scaling_diag_matrix is not full rank!")
            scaling_diag_matrix += 1e-6 * torch.eye(scaling_diag_matrix.shape[0]).to(dev)
            scaling_matrix_inv = torch.linalg.inv(scaling_diag_matrix)
                
        scaling_diag_matrix = scaling_diag_matrix.float()
        scaling_matrix_inv = scaling_matrix_inv.float()
        W_scale = torch.matmul(W, scaling_diag_matrix)
        
        U, S, VT = torch.linalg.svd(W_scale, full_matrices=False)
        
        num_s_after_trunc = int(W.shape[0] * W.shape[1] * ratio / (W.shape[0] + W.shape[1]))
        
        truc_s = S[:num_s_after_trunc]
        truc_u = U[:, :num_s_after_trunc]
        truc_v = torch.matmul(VT[:num_s_after_trunc, :], scaling_matrix_inv)
        truc_sigma = torch.diag(truc_s)
        
        #### Replace Attn, MLP ####
            
        sqrtSigma = torch.sqrt(truc_sigma)
        svd_u = torch.matmul(truc_u, sqrtSigma).cpu().to(dtype)
        svd_v = torch.matmul(sqrtSigma, truc_v).cpu().to(dtype)
        
        if "q_proj" in name:
            svd_attn.q_u_proj.weight.data = svd_u
            svd_attn.q_v_proj.weight.data = svd_v
        elif "k_proj" in name:
            svd_attn.k_u_proj.weight.data = svd_u
            svd_attn.k_v_proj.weight.data = svd_v
        elif "v_proj" in name:
            svd_attn.v_u_proj.weight.data = svd_u
            svd_attn.v_v_proj.weight.data = svd_v
        elif "o_proj" in name:
            svd_attn.o_u_proj.weight.data = svd_u
            svd_attn.o_v_proj.weight.data = svd_v
            layer.self_attn =  svd_attn
        elif "gate_proj" in name:
            svd_mlp.gate_u_proj.weight.data = svd_u
            svd_mlp.gate_v_proj.weight.data = svd_v
        elif "down_proj" in name:
            svd_mlp.down_u_proj.weight.data = svd_u
            svd_mlp.down_v_proj.weight.data = svd_v
        elif "up_proj" in name:
            svd_mlp.up_u_proj.weight.data = svd_u
            svd_mlp.up_v_proj.weight.data = svd_v
            layer.mlp = svd_mlp

100%|██████████| 32/32 [08:28<00:00, 15.88s/it]


In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): SVD_LlamaAttention(
          (q_u_proj): Linear(in_features=1638, out_features=4096, bias=False)
          (q_v_proj): Linear(in_features=4096, out_features=1638, bias=False)
          (k_u_proj): Linear(in_features=1638, out_features=4096, bias=False)
          (k_v_proj): Linear(in_features=4096, out_features=1638, bias=False)
          (v_u_proj): Linear(in_features=1638, out_features=4096, bias=False)
          (v_v_proj): Linear(in_features=4096, out_features=1638, bias=False)
          (o_u_proj): Linear(in_features=1638, out_features=4096, bias=False)
          (o_v_proj): Linear(in_features=4096, out_features=1638, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): SVD_LlamaMLP(
          (gate_u_proj): Linear(in_features=2388, out_features=11008, bias=False)
          (gate_v_p

In [ ]:
print(f'num params after SVD compression = {sum( [ np.prod(item.size())  for item in model.parameters() ])}')

num params after SVD compression = 5444636672


In [28]:
1103495168 / 1346471936

0.8195456128689785

In [29]:
model.to('cuda')

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): SVD_LlamaAttention(
          (q_u_proj): Linear(in_features=819, out_features=2048, bias=False)
          (q_v_proj): Linear(in_features=2048, out_features=819, bias=False)
          (k_u_proj): Linear(in_features=819, out_features=2048, bias=False)
          (k_v_proj): Linear(in_features=2048, out_features=819, bias=False)
          (v_u_proj): Linear(in_features=819, out_features=2048, bias=False)
          (v_v_proj): Linear(in_features=2048, out_features=819, bias=False)
          (o_u_proj): Linear(in_features=819, out_features=2048, bias=False)
          (o_v_proj): Linear(in_features=2048, out_features=819, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): SVD_LlamaMLP(
          (gate_u_proj): Linear(in_features=1194, out_features=5504, bias=False)
          (gate_v_proj): Lin

In [30]:
model.type(torch.float16)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): SVD_LlamaAttention(
          (q_u_proj): Linear(in_features=819, out_features=2048, bias=False)
          (q_v_proj): Linear(in_features=2048, out_features=819, bias=False)
          (k_u_proj): Linear(in_features=819, out_features=2048, bias=False)
          (k_v_proj): Linear(in_features=2048, out_features=819, bias=False)
          (v_u_proj): Linear(in_features=819, out_features=2048, bias=False)
          (v_v_proj): Linear(in_features=2048, out_features=819, bias=False)
          (o_u_proj): Linear(in_features=819, out_features=2048, bias=False)
          (o_v_proj): Linear(in_features=2048, out_features=819, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): SVD_LlamaMLP(
          (gate_u_proj): Linear(in_features=1194, out_features=5504, bias=False)
          (gate_v_proj): Lin

In [28]:
model.cuda().type(torch.bfloat16)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): SVD_LlamaAttention(
          (q_u_proj): Linear(in_features=1638, out_features=4096, bias=False)
          (q_v_proj): Linear(in_features=4096, out_features=1638, bias=False)
          (k_u_proj): Linear(in_features=1638, out_features=4096, bias=False)
          (k_v_proj): Linear(in_features=4096, out_features=1638, bias=False)
          (v_u_proj): Linear(in_features=1638, out_features=4096, bias=False)
          (v_v_proj): Linear(in_features=4096, out_features=1638, bias=False)
          (o_u_proj): Linear(in_features=1638, out_features=4096, bias=False)
          (o_v_proj): Linear(in_features=4096, out_features=1638, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): SVD_LlamaMLP(
          (gate_u_proj): Linear(in_features=2388, out_features=11008, bias=False)
          (gate_v_p

In [29]:
messages=[
    { 'role': 'user', 'content': "sort an array"}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
# tokenizer.eos_token_id is the id of <|EOT|> token
outputs = model.generate(inputs, use_cache = False, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


										<									
